In [33]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import logging
from gensim.models import word2vec
import gensim
import numpy as np

In [2]:
data = pd.read_csv('reviewComment.csv', delimiter=',')
data

,id,user_login,body,path,commit_id,in_reply_to_id,repo_full_name
0,121,agramfort,spaces after commas thanks :)\n,scikits/learn/svm/src/liblinear/_liblinear.pyx,e5291691809e5b4d35eb46af77680a85ebbe9976,0,NaN
1,122,agramfort,are you sure about cfort? is does not appear e...,scikits/learn/svm/src/liblinear/liblinear_help...,e5291691809e5b4d35eb46af77680a85ebbe9976,0,NaN
2,1161,elupus,Any reason this check is removed? Should we no...,xbmc/dbwrappers/Database.cpp,d153d81a509e00ecbc10d636ebbbd11ae7b562e5,0,NaN
3,1252,jmarshallnz,Yup - look up for internal_version (this inli...,xbmc/dbwrappers/Database.cpp,d153d81a509e00ecbc10d636ebbbd11ae7b562e5,0,NaN
4,1551,agramfort,does or does not?\n,doc/developers/neighbors.rst,25ef0935eea6de67c1574b61510c6715953733a1,0,NaN
...,...,...,...,...,...,...,...
99995,18391536,jamtur01,invoke\n,docs/sources/reference/builder.md,98ceae1a8499979f725bd04dfbc47ccf831331a7,0,NaN
99996,18391553,jamtur01,invoke\n,docs/sources/reference/builder.md,98ceae1a8499979f725bd04dfbc47ccf831331a7,0,NaN
99997,18391566,jamtur01,invoke\n,docs/sources/reference/builder.md,98ceae1a8499979f725bd04dfbc47ccf831331a7,0,NaN
99998,18391634,jamtur01,or `localhost`.\n,docs/sources/examples/running_ssh_service.md,1cc35466f288384fb5c0c8a75d45a8d56e840d27,0,NaN


In [3]:
# 去除HTML标签
def cleanHtml(subject):
    soup = BeautifulSoup(subject)
    str = [s.extract() for s in soup('pre')]
    
    return soup.text

In [4]:
data["body"] = data["body"].apply(cleanHtml)
data["body"]

c:\users\10065\anaconda3\envs\py3\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "http://en.wikipedia.org/wiki/Boolean
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
c:\users\10065\anaconda3\envs\py3\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "http://checkstyle.sourceforge.net/config_modifier.html
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
c:\users\10065\anaconda3\envs\py3\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "https://www.assembla.com/spaces/akka/tickets/1360-actorpath--add-rootpath-accessor-and-remove-string%28%29
" looks like a URL. Beautiful Soup is not an

c:\users\10065\anaconda3\envs\py3\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "http://cl.ly/1P37021y1w1V0y0o0v2m
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
c:\users\10065\anaconda3\envs\py3\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "http://www.assembla.com/spaces/akka/tickets/2228-handle-props-apply%5Bnothing%5D-as-props-empty
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
c:\users\10065\anaconda3\envs\py3\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "http://blogs.technet.com/b/grouppolicy/archive/2009/04/22/passwords-in-group-policy-preferences-updated.aspx
" lo

c:\users\10065\anaconda3\envs\py3\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "http://baystation12.net/forums/viewtopic.php?f=33&t=7850
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
c:\users\10065\anaconda3\envs\py3\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "http://dev.metasploit.com/redmine/issues/8083
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
c:\users\10065\anaconda3\envs\py3\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "http://en.wikipedia.org/wiki/Spurious_wakeup
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP clien

c:\users\10065\anaconda3\envs\py3\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "http://software.intel.com/sites/products/documentation/hpc/ipp/ippi/ippi_ch5/functn_MulC.html?
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
c:\users\10065\anaconda3\envs\py3\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "http://www.php.net/manual/en/function.wincache-ucache-set.php
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
c:\users\10065\anaconda3\envs\py3\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "https://github.com/rapid7/metasploit-framework/pull/3157/files#r11773568
" looks like 

0                          spaces after commas thanks :)\n
1        are you sure about cfort? is does not appear e...
2        Any reason this check is removed? Should we no...
3        Yup - look up  for internal_version (this inli...
4                                      does or does not?\n
                               ...                        
99995                                             invoke\n
99996                                             invoke\n
99997                                             invoke\n
99998                                    or `localhost`.\n
99999    Why the whitespace difference between no-cache...
Name: body, Length: 100000, dtype: object

In [5]:
# 去除->
def preClean(subject):
    beau = BeautifulSoup(subject)
    newSubject = beau.get_text()
    newSubject = newSubject.replace("->"," ")
    
    return newSubject

In [6]:
data["body"] = data["body"].apply(preClean)
data["body"]

0                          spaces after commas thanks :)\n
1        are you sure about cfort? is does not appear e...
2        Any reason this check is removed? Should we no...
3        Yup - look up  for internal_version (this inli...
4                                      does or does not?\n
                               ...                        
99995                                             invoke\n
99996                                             invoke\n
99997                                             invoke\n
99998                                    or `localhost`.\n
99999    Why the whitespace difference between no-cache...
Name: body, Length: 100000, dtype: object

In [7]:
# # 去除回复信息
# # 运行两次，因为一次头上的去不掉
# for cot in range(2):
#     for item in range(len(data)):
#         pattern_reply = re.compile(r'> (.*)\n')
#         temp = ""
#         if len(pattern_reply.findall(data["body"][item])) > 0:
#             temp = pattern_reply.sub("", data["body"][item])
#         pattern_reply_time = re.compile(r'On (.*)<\nreply@reply.github.com>(.*)wrote:')
#         if len(pattern_reply_time.findall(temp)) > 0:
#             temp = pattern_reply_time.sub("", temp)
#         if temp != "":
#             data["body"][item] = temp
# # data["body"]

In [8]:
# 去除回复信息
# 运行两次，因为一次头上的去不掉
# Version 2: reply格式
# 注意head和内容清洗顺序
for cot in range(2):
    for item in range(len(data)):
        
        temp = ""
                
#         pattern_reply_time4 = re.compile(r'\nFrom:([\s\S]*)\nSent:([\s\S]*)\nTo:([\s\S]*)\nCc:([\s\S]*)\nSubject:([\s\S]*)\n')
        pattern_reply_time4 = re.compile(r'\n([\s\S]*)From([\s\S]*):([\s\S]*)([\s\S]*)\n')
        if len(pattern_reply_time4.findall(data["body"][item])) > 0:
            temp = pattern_reply_time4.sub("", data["body"][item])
            
        if temp != "":
            data["body"][item] = temp
        
        pattern_reply_time5 = re.compile(r'\n([\s\S]*)De([\s\S]*):([\s\S]*)([\s\S]*)\n')
        if len(pattern_reply_time5.findall(data["body"][item])) > 0:
            temp = pattern_reply_time5.sub("", data["body"][item])
            
        if temp != "":
            data["body"][item] = temp
        
        
        pattern_reply_time3 = re.compile(r'\n([\s\S]*)@reply.github.com([\s\S]*)\n')
        if len(pattern_reply_time3.findall(data["body"][item])) > 0:
            temp = pattern_reply_time3.sub("", data["body"][item])
            
        if temp != "":
            data["body"][item] = temp
        
        pattern_reply_time = re.compile(r'On ([\s\S]*)wrote:')
        if len(pattern_reply_time.findall(data["body"][item])) > 0:
            temp = pattern_reply_time.sub("", data["body"][item])
            
        if temp != "":
            data["body"][item] = temp
            
        pattern_reply_time2 = re.compile(r'On (.*)<\nreply@reply.github.com>(.*)wrote:')
        
        if len(pattern_reply_time2.findall(data["body"][item])) > 0:
            temp = pattern_reply_time2.sub("", data["body"][item])
#         pattern_reply_time3 = re.compile(r'On (.*)<\nreply@reply.github.com>(.*)')
#         if len(pattern_reply_time.findall(temp)) > 0:
#             temp = pattern_reply_time3.sub("", temp)
        if temp != "":
            data["body"][item] = temp
            
#         pattern_reply = re.compile(r'> (.*)\n')
    
#         if len(pattern_reply.findall(data["body"][item])) > 0:
#             temp = pattern_reply.sub("", data["body"][item])

#         if temp != "":
#             data["body"][item] = temp
# data["body"]

c:\users\10065\anaconda3\envs\py3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\10065\anaconda3\envs\py3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\10065\anaconda3\envs\py3\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\10065\anaconda3\envs\py3\lib\site-packages\ipykernel_launcher.py:16

In [9]:
print(data["body"][38845])

how about just `autoencoder` or `auto_encoder`?  Except for the buzzword
value of the word _sparse_ does it really add much more? The only
non-sparse autoencoder I can think of is the bottleneck, _semantic hashing_
autoencoder which is specific enough to be called that if we ever implement
it. WDYT?



> In sklearn/neural_network/sae.py:
> 
> > +class SAE(BaseEstimator, TransformerMixin):
> > -    """
> > -    Sparse Autoencoder (SAE)
> >   +
> > -    A Sparse Autoencoder with one hidden layer. Parameters are trained using
> > -    Limited-Memory  Broyden-Fletcher-Goldfarb-Shanno (L-BFGS)
> > -    Parameters
> > -    ----------
> > -    n_hidden : int
> > -        Number of hidden neurons
> > -    lr : float, optional
> > -        Learning rate to use during learning. It is _highly_ recommended
> > -        to tune this hyper-parameter. Possible values are 10**[0., -3.].
> > -    beta : float, optional
> > -        Weight of sparsity penalty term
> > -    sparsityParam : float, optiona

In [10]:
# # 去除代码，打上代码标签
# for item in range(len(data)):
#     pattern = re.compile(r'```([\s\S]*)```')
#     if len(pattern.findall(data["body"][item])) > 0:
#         data["body"][item] = pattern.sub(data["path"][item].split('.')[-1].upper() + "CODE", data["body"][item])

In [11]:
print(data["body"][6342])
# data["body"]

I know. The reviews are good opportunity to cleanup whatever is noticed, that's the way we roll all the time.


In [12]:
print(data["body"][101])
# data["body"]

alias   ...   alias   service

```
public function get($id, $invalidBehavior = ContainerInterface::EXCEPTION_ON_INVALID_REFERENCE)
{
....
        if (!$this hasDefinition($id) && isset($this aliases[$id])) {
            return $this get($this aliases[$id]);
        }
....
}
```



In [13]:
# 预处理
def cleanReview(subject):
    beau = BeautifulSoup(subject)
    newSubject = beau.get_text()
    newSubject = newSubject.replace("--"," ")
    newSubject = newSubject.replace("..."," ").replace("…"," ")
    newSubject = newSubject.replace("\\", "").replace("\'","").replace('/', '').replace('"', '').replace(',', ' ').replace('.', ' ').replace('?', ' ').replace('!',' ').replace('(', ' ').replace(')', ' ').replace(':',' ')
    newSubject = newSubject.strip().split(" ")
    newSubject = [word.lower() for word in newSubject]
    # join() 方法用于将序列中的元素以指定的字符连接生成一个新的字符串
    newSubject = " ".join(newSubject)
    
    return newSubject

In [14]:
data["body"] = data["body"].apply(cleanReview)
data["body"]

0                               spaces after commas thanks
1        are you sure about cfort  is does not appear e...
2        any reason this check is removed  should we no...
3        yup - look up  for internal_version  this inli...
4                                         does or does not
                               ...                        
99995                                               invoke
99996                                               invoke
99997                                               invoke
99998                                       or `localhost`
99999    why the whitespace difference between no-cache...
Name: body, Length: 100000, dtype: object

In [15]:
print(data["body"][668])

i think that it is perfectly valid to only implement a decoder  this would


In [16]:
# data["body"][2]

In [17]:
# 去除代码，打上代码标签
for item in range(len(data)):
    pattern = re.compile(r'```([\s\S]*)```')
    if len(pattern.findall(data["body"][item])) > 0:
        data["body"][item] = pattern.sub(data["path"][item].split('.')[-1].upper() + "CODE", data["body"][item])

c:\users\10065\anaconda3\envs\py3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [18]:
# 去除`包围的变量名
for item in range(len(data)):
    pattern = re.compile(r'`([\s\S]*)`')
    if len(pattern.findall(data["body"][item])) > 0:
        data["body"][item] = pattern.sub("VAR", data["body"][item])

c:\users\10065\anaconda3\envs\py3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [19]:
data["body"][101]

'alias       alias   service\n\nPHPCODE'

In [20]:
# data["path"][10].split('.')[-1].upper()

In [21]:
# 去除>
pattern_reply = re.compile(r'>(.*)\n')

for item in range(len(data)):
    temp = ""
    if len(pattern_reply.findall(data["body"][item])) > 0:
        temp = pattern_reply.sub("", data["body"][item])

    if temp != "":
        data["body"][item] = temp

c:\users\10065\anaconda3\envs\py3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [22]:
print(data["body"][38845])

how about just VAR   except for the buzzword
value of the word _sparse_ does it really add much more  the only
non-sparse autoencoder i can think of is the bottleneck  _semantic hashing_
autoencoder which is specific enough to be called that if we ever implement
it  wdyt 



>


In [23]:
# 去除\n\r
def cleanToOneLine(subject):
    beau = BeautifulSoup(subject)
    newSubject = beau.get_text()
    newSubject = newSubject.replace('\n',' ').replace('\r',' ')
    
    return newSubject

In [ ]:
# 去除停用词
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
#英文停止词，set()集合函数消除重复项
    list_stopWords=list(set(stopwords.words('english')))
    example_text="Five score years ago, a great American, in whose symbolic shadow we stand today, signed the Emancipation Proclamation. This momentous decree came as a great beacon light of hope to millions of Negro slaves who had been seared in the flames of withering injustice. It came as a joyous daybreak to end the long night of bad captivity."
    #分句
    list_sentences=sent_tokenize(example_text)
    #分词
    list_words=word_tokenize(example_text)
    #过滤停止词
    filtered_words=[w for w in list_words if not w in list_stopWords]

In [24]:
data["body"] = data["body"].apply(cleanToOneLine)

In [25]:
for i in range(100,200):
    print(i," : ",data["body"][i])

100  :  this is what the history of the commit on the file is for  we dont want to have 5 loc and 50 comment lines when a line is changed several times
101  :  alias       alias   service  PHPCODE
102  :  thats typically a method that should now be private ;
103  :  check isset   for both query and fields - getting undefined index
104  :  ha  ok  so im going in the wrong direction  i should be able to accomplish some of the same goals by using only the VAR is important
105  :  ah yes  youre most definitely right  if im allowed to stick with using the VAR  ill make sure to include this
106  :  i fail to see how your code is equivalent to the old one
107  :  you are right  it is not equivalent  there is no need to build and url  which will certainly be converted back to a filename at some point    i forgot to mention that this has been validated on both win & linux
108  :  why are you using a dash everywhere   this is not correct here
109  :  it should be VAR  not an hardcoded 5
110  :  

In [26]:
print(data["body"][101])

alias       alias   service  PHPCODE


In [27]:
for i in range(len(data)):
    if(data["body"][i].find('logisticregression') >= 0):
        print(i," : ",data["body"][i])

44419  :  lets support cpp version only  we dont want to care about outdated c style api  - logisticregression  the same for the params of lr
44420  :  logisticregressiontrainparams looks better than logisticregression_trainparams
44542  :  sorry  sure  ill try to be more specific   cv prefix is c style of opencv  cv   namespace is c++  so rename cvlr   logisticregression  cvlr_trainparams   logisticregressiontrainparams and place them in cv   namespace  use cv  termcriteria instead of cvtermcriteria  line 1999    in the implementation below there are cvmat  cvreadintbyname     and other similar c functions   btw there is one example of c++ ml model in opencv  see cv  em  its inherited from cv  algorithm  but i think we may not inherit from cv  algoritm because this class is changing now  for coming opencv 3 0   we also should not inherit from cvstatmodel and should replace current readwrite methods by the same api like cv  algorithm has   virtual void write filestorage& fs  const; vir

In [28]:
newDf = data["body"]
newDf.to_csv("cleaned.txt",index=False)

c:\users\10065\anaconda3\envs\py3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [29]:
# 设置输出日志
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# 直接用gemsim提供的API去读取txt文件，读取文件的API有LineSentence 和 Text8Corpus, PathLineSentences等。
sentences = word2vec.LineSentence("cleaned.txt")

# 训练模型，词向量的长度设置为200， 迭代次数为8，采用skip-gram模型，模型保存为bin格式
model = gensim.models.Word2Vec(sentences, size=200, sg=1, iter=8)  
model.wv.save_word2vec_format("./word2Vec" + ".bin", binary=True) 

# 加载bin格式的模型
wordVec = gensim.models.KeyedVectors.load_word2vec_format("word2Vec.bin", binary=True)

2021-01-12 15:40:38,836 : INFO : collecting all words and their counts
2021-01-12 15:40:38,844 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-01-12 15:40:38,896 : INFO : PROGRESS: at sentence #10000, processed 161261 words, keeping 11124 word types
2021-01-12 15:40:38,952 : INFO : PROGRESS: at sentence #20000, processed 335897 words, keeping 18083 word types
2021-01-12 15:40:39,008 : INFO : PROGRESS: at sentence #30000, processed 502445 words, keeping 23493 word types
2021-01-12 15:40:39,064 : INFO : PROGRESS: at sentence #40000, processed 676062 words, keeping 28670 word types
2021-01-12 15:40:39,120 : INFO : PROGRESS: at sentence #50000, processed 853359 words, keeping 33490 word types
2021-01-12 15:40:39,178 : INFO : PROGRESS: at sentence #60000, processed 1026842 words, keeping 37771 word types
2021-01-12 15:40:39,234 : INFO : PROGRESS: at sentence #70000, processed 1192122 words, keeping 41645 word types
2021-01-12 15:40:39,289 : INFO : PROGRESS: a

In [30]:
wordVec = gensim.models.KeyedVectors.load_word2vec_format("word2Vec.bin", binary=True)
wordVec['-']

2021-01-12 15:41:03,476 : INFO : loading projection weights from word2Vec.bin
2021-01-12 15:41:03,526 : INFO : loaded (9672, 200) matrix from word2Vec.bin


array([ 0.21260378,  0.09926204, -0.42197123,  0.07171463, -0.11635962,
       -0.1587817 , -0.22813904, -0.40599933,  0.10550229, -0.02989995,
        0.29662153,  0.07556977, -0.31014094,  0.1351672 ,  0.5043207 ,
        0.1870861 ,  0.02517163, -0.21897951, -0.01298134,  0.03930113,
       -0.27110675, -0.14548273,  0.19351892,  0.607968  , -0.38757592,
       -0.07881724,  0.07387956,  0.32218164, -0.02288199,  0.18185848,
       -0.15622565, -0.23542303,  0.01394012, -0.01565598, -0.02710978,
       -0.1132633 ,  0.23187968,  0.02718084, -0.49805453,  0.15765472,
       -0.13148794, -0.07154568,  0.0538125 ,  0.18598424,  0.1474826 ,
        0.25623986,  0.17495002, -0.28342375,  0.13444032,  0.2659585 ,
        0.06289336, -0.12813604,  0.24312884, -0.03483532,  0.08115482,
        0.32567513,  0.10803639, -0.55382454, -0.32812965, -0.06610359,
       -0.46951994, -0.05389235, -0.16940878,  0.19074863,  0.34751576,
        0.21043657, -0.11064307,  0.11307571, -0.08642122,  0.08

In [40]:
# 和某个单词关系最近的10个单向量
wordVec.most_similar(['actor'])

[('supervisor', 0.7129275798797607),
 ('actors', 0.6895064115524292),
 ('actorref', 0.6859934329986572),
 ('actorsystem', 0.6632527112960815),
 ('restarted', 0.6331169009208679),
 ('endpoint', 0.6227724552154541),
 ('terminates', 0.6139692068099976),
 ('actorcell', 0.6136939525604248),
 ('uid', 0.6107230186462402),
 ('coordinator', 0.6084626913070679)]

In [34]:
# 计算句子向量
sentences = []
a = np.zeros(200, dtype=np.float32)
for s in data["body"]:
    for word in s.split(' '):
        try:  
            wordv = wordVec[word]
        except:
            wordv = np.zeros(200, dtype=np.float32)
        a = a + wordv
        a = a * [0.5]
    sentences.append(a)
    a = np.zeros(200, dtype=np.float32)
# print(sentences)

In [35]:
sentences[0]

array([ 0.02774239,  0.16575766, -0.31319333,  0.17946357,  0.02485174,
       -0.16075351, -0.06282411, -0.09537544,  0.01586559, -0.00509875,
       -0.27884047,  0.32087677, -0.01700382,  0.16575594, -0.19591502,
        0.40138265,  0.2075412 ,  0.12191047,  0.15799911, -0.04422868,
       -0.32020175,  0.01320735,  0.25614699, -0.24586379, -0.49924347,
       -0.41935074,  0.08263077,  0.05711561, -0.15952523, -0.06189486,
       -0.13357191,  0.33550413, -0.0649945 ,  0.14911127, -0.14400562,
       -0.5227334 ,  0.28935875, -0.28232474, -0.0823628 ,  0.09847927,
       -0.23040136, -0.11467727, -0.10678585, -0.12252446,  0.03523265,
        0.03419405,  0.13339036, -0.20008489,  0.26864238, -0.1545718 ,
        0.12447099, -0.19348758,  0.26327304, -0.31910456,  0.04575398,
       -0.29995403,  0.23620575,  0.24094101, -0.06088914, -0.12883687,
       -0.45787011, -0.37160794, -0.06510864,  0.11160031, -0.04689757,
        0.10600913,  0.10576841,  0.43407408, -0.01146152,  0.08

In [36]:
np.save("sentences",sentences)

In [37]:
sentences = np.load("sentences.npy")
sentences[0]

array([ 0.02774239,  0.16575766, -0.31319333,  0.17946357,  0.02485174,
       -0.16075351, -0.06282411, -0.09537544,  0.01586559, -0.00509875,
       -0.27884047,  0.32087677, -0.01700382,  0.16575594, -0.19591502,
        0.40138265,  0.2075412 ,  0.12191047,  0.15799911, -0.04422868,
       -0.32020175,  0.01320735,  0.25614699, -0.24586379, -0.49924347,
       -0.41935074,  0.08263077,  0.05711561, -0.15952523, -0.06189486,
       -0.13357191,  0.33550413, -0.0649945 ,  0.14911127, -0.14400562,
       -0.5227334 ,  0.28935875, -0.28232474, -0.0823628 ,  0.09847927,
       -0.23040136, -0.11467727, -0.10678585, -0.12252446,  0.03523265,
        0.03419405,  0.13339036, -0.20008489,  0.26864238, -0.1545718 ,
        0.12447099, -0.19348758,  0.26327304, -0.31910456,  0.04575398,
       -0.29995403,  0.23620575,  0.24094101, -0.06088914, -0.12883687,
       -0.45787011, -0.37160794, -0.06510864,  0.11160031, -0.04689757,
        0.10600913,  0.10576841,  0.43407408, -0.01146152,  0.08

In [39]:
data.to_csv('data_cleaned.csv')